In [1]:
import tkinter as tk
from tkinter import messagebox, ttk
import csv
from datetime import datetime

expenses = []
budget = 0

def load_expenses():
    global expenses
    try:
        with open("expenses.csv", "r") as file:
            reader = csv.DictReader(file)
            expenses = [row for row in reader]
            for expense in expenses:
                expense["amount"] = float(expense["amount"])
    except FileNotFoundError:
        expenses = []

def save_expenses():
    with open("expenses.csv", "w", newline="") as file:
        fieldnames = ["date", "category", "amount", "description"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(expenses)
    messagebox.showinfo("Save Expenses", "Expenses saved successfully!")

def add_expense():
    def submit_expense():
        date = date_entry.get()
        category = category_entry.get()
        try:
            amount = float(amount_entry.get())
        except ValueError:
            messagebox.showerror("Invalid Input", "Amount must be a number!")
            return
        description = description_entry.get()

        if not (date and category and description):
            messagebox.showerror("Invalid Input", "All fields are required!")
            return

        try:
            datetime.strptime(date, "%Y-%m-%d")
        except ValueError:
            messagebox.showerror("Invalid Input", "Date must be in YYYY-MM-DD format!")
            return

        expenses.append({
            "date": date,
            "category": category,
            "amount": amount,
            "description": description,
        })
        messagebox.showinfo("Add Expense", "Expense added successfully!")
        add_window.destroy()

    add_window = tk.Toplevel(root)
    add_window.title("Add Expense")

    tk.Label(add_window, text="Date (YYYY-MM-DD):").pack()
    date_entry = tk.Entry(add_window)
    date_entry.pack()

    tk.Label(add_window, text="Category:").pack()
    category_entry = tk.Entry(add_window)
    category_entry.pack()

    tk.Label(add_window, text="Amount:").pack()
    amount_entry = tk.Entry(add_window)
    amount_entry.pack()

    tk.Label(add_window, text="Description:").pack()
    description_entry = tk.Entry(add_window)
    description_entry.pack()

    tk.Button(add_window, text="Submit", command=submit_expense).pack()


def view_expenses():
    view_window = tk.Toplevel(root)
    view_window.title("View Expenses")

    tree = ttk.Treeview(view_window, columns=("date", "category", "amount", "description"), show="headings")
    tree.heading("date", text="Date")
    tree.heading("category", text="Category")
    tree.heading("amount", text="Amount")
    tree.heading("description", text="Description")
    tree.pack(fill=tk.BOTH, expand=True)

    for expense in expenses:
        tree.insert("", tk.END, values=(expense["date"], expense["category"], expense["amount"], expense["description"]))

def set_budget():
    def submit_budget():
        global budget
        try:
            budget = float(budget_entry.get())
            messagebox.showinfo("Set Budget", f"Budget set to {budget:.2f}")
            budget_window.destroy()
        except ValueError:
            messagebox.showerror("Invalid Input", "Budget must be a number!")

    budget_window = tk.Toplevel(root)
    budget_window.title("Set Budget")

    tk.Label(budget_window, text="Enter Monthly Budget:").pack()
    budget_entry = tk.Entry(budget_window)
    budget_entry.pack()

    tk.Button(budget_window, text="Submit", command=submit_budget).pack()

def track_budget():
    total_expenses = sum(expense["amount"] for expense in expenses)
    if total_expenses > budget:
        messagebox.showwarning("Budget Warning", f"You have exceeded your budget by {total_expenses - budget:.2f}!")
    else:
        messagebox.showinfo("Budget Tracker", f"You have {budget - total_expenses:.2f} left for the month.")

def process_menu_choice():
    choice = menu_entry.get()
    if choice == "1":
        add_expense()
    elif choice == "2":
        view_expenses()
    elif choice == "3":
        set_budget()
    elif choice == "4":
        track_budget()
    elif choice == "5":
        save_expenses()
    elif choice == "6":
        save_expenses()
        root.destroy()
    else:
        messagebox.showerror("Invalid Choice", "Please select a valid option!")

root = tk.Tk()
root.title("Personal Expense Tracker")

tk.Label(root, text="Personal Expense Tracker Menu", font=("Arial", 16)).pack(pady=10)
tk.Label(root, text="1. Add Expense").pack()
tk.Label(root, text="2. View Expenses").pack()
tk.Label(root, text="3. Set Budget").pack()
tk.Label(root, text="4. Track Budget").pack()
tk.Label(root, text="5. Save Expenses").pack()
tk.Label(root, text="6. Exit").pack()

tk.Label(root, text="Enter your choice:").pack(pady=5)
menu_entry = tk.Entry(root)
menu_entry.pack()

tk.Button(root, text="Submit", command=process_menu_choice).pack(pady=10)

load_expenses()
root.mainloop()
